# **THE AIM HERE IS TO CHECK WHETHER USING GRAPHS INSTEAD OF EMBEDDINGS CAN RESULT IN A BETTER RAG SYSTEM OR NOT**

 **WE USE DIFFBOT TO EXTRACT THE ENTITES AND RELATIONSHIPS FROM THE RAW TEXT USING ITS FREE API AND THEN USE A NEO4J INSTANCE TO QUERY IT**

In [1]:
!pip install langchain-community langchain langchain-experimental langchain-groq neo4j wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer

diffbot_api_key = "YOUR_API_KEY"
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

In [3]:
from langchain_community.document_loaders import WikipediaLoader

query = "Warren Buffett"
raw_documents = WikipediaLoader(query=query).load()

In [4]:
graph_documents = diffbot_nlp.convert_to_graph_documents(raw_documents)

In [6]:
from langchain_community.graphs import Neo4jGraph

url = "bolt://18.233.152.220:7687"
username = "neo4j"
password = "escapes-signalman-flares"

graph = Neo4jGraph(url=url, username=username, password=password)

In [7]:
graph.add_graph_documents(graph_documents)

In [8]:
graph.refresh_schema()

In [9]:
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq

chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768"),
    qa_llm=ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768"),
    graph=graph,
    verbose=True,
)

In [10]:
chain.invoke("Who is or was working at Berkshire Hathaway?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:EMPLOYEE_OR_MEMBER_OF]->(o:Organization)
WHERE o.name = 'Berkshire Hathaway'
RETURN p.name
Full Context:
[{'p.name': 'David Gottesman'}, {'p.name': 'Michael Goldberg'}, {'p.name': 'Ajit Jain'}, {'p.name': 'Gregory E. Abel'}, {'p.name': 'Charlie Munger'}, {'p.name': 'Oliver Chace'}, {'p.name': 'Howard Buffett'}, {'p.name': 'Howard'}, {'p.name': 'Susan Buffett'}, {'p.name': 'Warren Buffett'}]

> Finished chain.


{'query': 'Who is or was working at Berkshire Hathaway?',
 'result': 'David Gottesman, Michael Goldberg, Ajit Jain, Gregory E. Abel, Charlie Munger, Oliver Chace, Howard Buffett, Howard, Susan Buffett, and Warren Buffett have or had a connection to Berkshire Hathaway.'}

# **USING CHROMA DB TO STORE THE EMBEDDIBGS AND QUERYING IT USING SIMILARITY SEARCH**

In [16]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6

In [17]:
# import
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter


# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(raw_documents)

# create the open-source embedding function
embedding_function = OllamaEmbeddings(base_url= "https://viper-intent.ngrok-free.app", model="nomic-embed-text")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
query = "Who is or was working at Berkshire Hathaway?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

== Career ==
From 1973 to 1976, Jain worked for IBM as a salesman for their data-processing operations in India. He was named "Rookie of the Year" in his region in 1973. He lost his job in 1976 when IBM discontinued their operation in India because they declined to allow any Indian ownership of the company, as was then required by law.
In 1978, Jain moved to the United States, where he earned an MBA from Harvard University and joined McKinsey & Co. He returned to India in the early 1980s and married. The Jains then moved back to the United States, as Jain's wife preferred to live there.
In 1986, he left McKinsey to work on insurance operations for Warren Buffett. Jain was invited by his former boss, Michael Goldberg, who had left McKinsey & Co. to join Berkshire Hathaway in 1982. At the time, he said he knew little about the insurance business.
In the annual letter to shareholders on 2014, it was suggested that both Jain and Greg Abel could be appropriate successors for Warren Buffett 

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768")

system = "You are a helpful assistant. Answer very precisely and concisely according to the question"
human = f"{text}  Relevant content {docs[0].page_content}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Who is or was working at Berkshire Hathaway? I need names"})

AIMessage(content='The relevant career details of the person are:\n\n1. Worked as a salesman for IBM\'s data-processing operations in India from 1973 to 1976, and was named "Rookie of the Year" in his region in 1973.\n2. Lost his job at IBM in 1976 when they discontinued their operation in India.\n3. Earned an MBA from Harvard University and joined McKinsey & Co. in 1978.\n4. Returned to India in the early 1980s and got married.\n5. Left McKinsey in 1986 to work on insurance operations for Warren Buffett, invited by his former boss, Michael Goldberg.\n6. In 2014, suggested as a potential successor for Warren Buffett as CEO of Berkshire Hathaway.\n7. Named Berkshire Hathaway\'s vice chair of insurance operations and appointed to Hathaway\'s board of directors in January 2018.', response_metadata={'token_usage': {'completion_tokens': 233, 'prompt_tokens': 355, 'total_tokens': 588, 'completion_time': 0.395368697, 'prompt_time': 0.064974772, 'queue_time': None, 'total_time': 0.460343469}, 